In [1]:
import numpy as np
import os
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# 读取数据
load_path = '../global_data/time_76800x128x9x9/'

trials = np.load(load_path + 'trials.npy')
bases = np.load(load_path + 'bases.npy')
labels = np.load(load_path + 'labels.npy')
# chw格式
print(trials.shape, bases.shape, labels.shape)

(76800, 128, 9, 9) (1280, 128, 9, 9) (1280, 2)


## 数据预处理

In [3]:
# 去基线
trials_de_base = []
for i, base in enumerate(bases):
    trials_de_base.append(trials[i * 60 : (i + 1) * 60] - base)
trials_de_base = np.array(trials_de_base)
trials_de_base = trials_de_base.reshape((-1, 128, 9, 9))
trials_de_base.shape

(76800, 128, 9, 9)

In [4]:
# 离散化标签
labels = np.where(labels >= 5, 1, 0)

# 复制以对齐样本
labels = np.repeat(labels, 60, axis = 0)
print(labels.shape)

(76800, 2)


In [5]:
def data_split(features, labels, train_ratio = 0.9):
    sample_cnt = labels.shape[0]
    # 数据集划分
    train_ratio = train_ratio
    shuffer_list = np.arange(sample_cnt)
    np.random.shuffle(shuffer_list)

    features_shuffer = features[shuffer_list] # trials_de_base
    labels_shuffer = labels[shuffer_list]

    x_train = features_shuffer[:int(train_ratio * sample_cnt)]
    y_train = labels_shuffer[:int(train_ratio * sample_cnt)]
    x_test = features_shuffer[int(train_ratio * sample_cnt):]
    y_test = labels_shuffer[int(train_ratio * sample_cnt):]
    return x_train, y_train, x_test, y_test

In [6]:
def z_norm(x_train, x_test):# z归一化
    chan_to_1020={0:[0,3],1:[1,3],2:[2,2],3:[2,0],4:[3,1],5:[3,3],6:[4,2],7:[4,0],8:[5,1],
                  9:[5,3],10:[6,2],11:[6,0],12:[7,3],13:[8,3],14:[8,4],15:[6,4],16:[0,5],
                  17:[1,5],18:[2,4],19:[2,6],20:[2,8],21:[3,7],22:[3,5],23:[4,4],24:[4,6],
                    25:[4,8],26:[5,7],27:[5,5],28:[6,6],29:[6,8],30:[7,5],31:[8,5]}

    # 需要归一化的位置
    need_norm_pos = np.zeros(shape=(9,9), dtype=int)

    for val in chan_to_1020.values():
        need_norm_pos[val[0]][val[1]] = 1
    # print(need_norm_pos)

    for i in range(9):
        for j in range(9):
            if need_norm_pos[i][j] == 0:
                continue
            for t in range(128):
                mean = np.mean(x_train[:, t, i, j])
                std = np.std(x_train[:, t, i, j])
                x_train[:, t, i, j] -= mean
                x_train[:, t, i, j] /= std
                x_test[:, t, i, j] -= mean
                x_test[:, t, i, j] /= std
    return x_train, x_test

In [7]:
# 超参设置
batch_size = 32
max_epoch = 200
emotion_dim = 0

patient = 10

model_save_path = 'model/transformer_model/'

In [8]:
from torch.utils.data import TensorDataset
import torch
from torch.utils.data import DataLoader

In [9]:
import math
from IndiviTransNet import IndiviTransNet

for sub_idx in range(32):
    
    model = IndiviTransNet(transformer_phase=False)
    # 初始化权重参数
    for layer in model.modules():
        if isinstance(layer, nn.Linear):
            nn.init.xavier_uniform_(layer.weight)

    opt = torch.optim.Adam(model.parameters(), 0.001)
    criterion = nn.CrossEntropyLoss()

    train_loss_holder = []
    vali_loss_holder = []

    # 提前终止
    acc_value=0
    step=0
    
    sub_feature = trials_de_base[sub_idx * 2400 : (sub_idx + 1) * 2400]
    sub_labels = labels[sub_idx * 2400 : (sub_idx + 1) * 2400]
    x_train, y_train, x_test, y_test = data_split(sub_feature, sub_labels,train_ratio=0.9)
    
    x_train, x_test = z_norm(x_train, x_test)
    y_train = y_train[:, emotion_dim]
    y_test = y_test[:, emotion_dim]
    
    x_train = torch.tensor(x_train, dtype=torch.float)
    y_train = torch.tensor(y_train, dtype=torch.long)
    x_test = torch.tensor(x_test, dtype=torch.float)
    y_test = torch.tensor(y_test, dtype=torch.long)
    
    data_train = TensorDataset(x_train, y_train)
#     data_test = TensorDataset(x_test, y_test)
    
    train_loader = DataLoader(dataset=data_train, batch_size=batch_size, shuffle=True)
#     test_loader = DataLoader(dataset=data_test, batch_size=batch_size, shuffle=True)
    
    for i in range(max_epoch):
        model.train()
        for batch_idx, batch_data in enumerate(train_loader):
            x, y = batch_data
            _, outputs = model(x)
            loss = criterion(outputs, y)
            opt.zero_grad()  
            loss.backward()
            opt.step()
        
        model.eval()
        _, outputs = model(x_train)
        train_loss = criterion(outputs, y_train) / x_train.shape[0]
        predicted = torch.max(outputs, 1)[1]
        train_acc = (predicted == y_train).sum() / x_train.shape[0]
        
        _, outputs = model(x_test)
        vali_loss = criterion(outputs, y_test)  / x_test.shape[0]
        predicted = torch.max(outputs, 1)[1]
        vali_acc = (predicted == y_test).sum() / x_test.shape[0]
        print(f'epoch: {i}, train_loss: {train_loss:.6f}, train_acc: {train_acc:.6f}, vali_loss: {vali_loss:.6f}, vali_acc: {vali_acc:.6f}')
        
        step = step + 1
        #模型性能有所提升则保存模型，并更新loss_value
        if vali_acc > acc_value:
            step = 0
            torch.save(model.state_dict(), model_save_path + f'sub_{sub_idx}_model_parameter.pkl')
            acc_value = vali_acc
        
        if step >= patient:
            break
    


epoch: 0, train_loss: 0.000006, train_acc: 0.995833, vali_loss: 0.000192, vali_acc: 0.987500
epoch: 1, train_loss: 0.000002, train_acc: 0.999074, vali_loss: 0.000073, vali_acc: 0.991667
epoch: 2, train_loss: 0.000007, train_acc: 0.994907, vali_loss: 0.000434, vali_acc: 0.975000
epoch: 3, train_loss: 0.000001, train_acc: 0.998611, vali_loss: 0.000146, vali_acc: 0.987500
epoch: 4, train_loss: 0.000006, train_acc: 0.995370, vali_loss: 0.000131, vali_acc: 0.991667
epoch: 5, train_loss: 0.000009, train_acc: 0.993519, vali_loss: 0.000378, vali_acc: 0.975000
epoch: 6, train_loss: 0.000003, train_acc: 0.997685, vali_loss: 0.000191, vali_acc: 0.983333
epoch: 7, train_loss: 0.000004, train_acc: 0.996759, vali_loss: 0.000224, vali_acc: 0.979167
epoch: 8, train_loss: 0.000004, train_acc: 0.996759, vali_loss: 0.000206, vali_acc: 0.991667
epoch: 9, train_loss: 0.000014, train_acc: 0.992130, vali_loss: 0.000504, vali_acc: 0.979167
epoch: 10, train_loss: 0.000001, train_acc: 0.999537, vali_loss: 0.000

#### TODO: 用第一个人的网络，测试第二个人，性能怎么样？
    如果性能不好，就说明自己用自己的网络性能好，所以在训练多任务网络时，就应该鼓励受试者自己使用自己的网络
#### 经过证明，性能确实不好，在40-60之间

In [10]:
# 测试，使用其他人的模型，性能是否会好
# 第 i 个受试者，使用第（i+1）%32 个模型

import math
from IndiviTransNet import IndiviTransNet

for sub_idx in range(32):
    sub_feature = trials_de_base[sub_idx * 2400 : (sub_idx + 1) * 2400]
    sub_labels = labels[sub_idx * 2400 : (sub_idx + 1) * 2400]
    x_train, y_train, x_test, y_test = data_split(sub_feature, sub_labels,train_ratio=0.9)
    
    x_train, x_test = z_norm(x_train, x_test)
    y_train = y_train[:, emotion_dim]
    y_test = y_test[:, emotion_dim]
    
    x_train = torch.tensor(x_train, dtype=torch.float)
    y_train = torch.tensor(y_train, dtype=torch.long)
    x_test = torch.tensor(x_test, dtype=torch.float)
    y_test = torch.tensor(y_test, dtype=torch.long)
    
    model = IndiviTransNet(transformer_phase=True)
    model.load_state_dict(torch.load(model_save_path + f'sub_{(sub_idx + 1) % 32}_model_parameter.pkl'))
    model.eval()
    _, outputs = model(x_train)
    train_loss = criterion(outputs, y_train) / x_train.shape[0]
    predicted = torch.max(outputs, 1)[1]
    train_acc = (predicted == y_train).sum() / x_train.shape[0]
    
    _, outputs = model(x_test)
    vali_loss = criterion(outputs, y_test)  / x_test.shape[0]
    predicted = torch.max(outputs, 1)[1]
    vali_acc = (predicted == y_test).sum() / x_test.shape[0]
    print(f'sub: {sub_idx}, train_loss: {train_loss:.6f}, train_acc: {train_acc:.6f}, vali_loss: {vali_loss:.6f}, vali_acc: {vali_acc:.6f}')

sub: 0, train_loss: 0.002910, train_acc: 0.463889, vali_loss: 0.025143, vali_acc: 0.445833
sub: 1, train_loss: 0.000891, train_acc: 0.487963, vali_loss: 0.007562, vali_acc: 0.500000
sub: 2, train_loss: 0.001586, train_acc: 0.503704, vali_loss: 0.013377, vali_acc: 0.537500
sub: 3, train_loss: 0.000901, train_acc: 0.417593, vali_loss: 0.007550, vali_acc: 0.437500
sub: 4, train_loss: 0.001126, train_acc: 0.607870, vali_loss: 0.012199, vali_acc: 0.558333
sub: 5, train_loss: 0.001175, train_acc: 0.660648, vali_loss: 0.009074, vali_acc: 0.666667
sub: 6, train_loss: 0.000872, train_acc: 0.511574, vali_loss: 0.007767, vali_acc: 0.545833
sub: 7, train_loss: 0.001048, train_acc: 0.549074, vali_loss: 0.009751, vali_acc: 0.520833
sub: 8, train_loss: 0.001526, train_acc: 0.459259, vali_loss: 0.014557, vali_acc: 0.437500
sub: 9, train_loss: 0.000657, train_acc: 0.466204, vali_loss: 0.005946, vali_acc: 0.420833
sub: 10, train_loss: 0.000975, train_acc: 0.494907, vali_loss: 0.008098, vali_acc: 0.49166

In [11]:
# os.environ["PATH"] += os.pathsep +'C:/Program Files/Graphviz/bin'
# plot_model(model,'cnn2_dens2.jpg', show_shapes=True)

In [12]:
# # 建立模型


# callbacksList = [EarlyStopping(monitor = 'val_acc', patience=10, verbose = 1), # patience当连续多少个epochs时验证集精度不再变好终止训练
#                 ModelCheckpoint(filepath = f'model/model.h5',monitor='acc',save_best_only=True,)
# ]

# # 评估
# model = load_model('model/model.h5')
# score = model.evaluate(x_test, y__v_test, verbose = 1)

# """
# Epoch 39/200
# 2160/2160 [==============================] - 59s 27ms/step - loss: 0.0448 - acc: 0.9878 - val_loss: 0.5709 - val_acc: 0.9161
# """